In [80]:
import os
import time
import pickle
import gc
import glob, re
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import datetime 

In [81]:
BASE_DIR = '/home/maddy14awasthi/DS/recruit-restaurant-visitor-forecasting'

# A naive time function
def time_elapsed(t0, str_result=True):
    t = np.round((time.time()-t0)/60, 3)
    if str_result==True:
        return str(t)+' minutes elapsed!'
    else:
        return t

In [82]:
data = {'tra': pd.read_csv(os.path.join(BASE_DIR, 'data', 'air_visit_data.csv.zip')), 
        'as': pd.read_csv(os.path.join(BASE_DIR, 'data', 'air_store_info.csv.zip')),
        'hs': pd.read_csv(os.path.join(BASE_DIR, 'data', 'hpg_store_info.csv.zip')),
        'ar': pd.read_csv(os.path.join(BASE_DIR, 'data', 'air_reserve.csv.zip')),
        'hr': pd.read_csv(os.path.join(BASE_DIR, 'data', 'hpg_reserve.csv.zip')),
        'id': pd.read_csv(os.path.join(BASE_DIR, 'data', 'store_id_relation.csv.zip')),
        'tes': pd.read_csv(os.path.join(BASE_DIR, 'data', 'sample_submission.csv.zip')),
        'hol': pd.read_csv(os.path.join(BASE_DIR, 'data', 'date_info.csv.zip')).rename(columns={'calendar_date':'visit_date'})}

data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])

In [83]:
for df in ['ar','hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r: (r['visit_datetime']-r['reserve_datetime']).days, axis=1)
    tmp1 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[[
           'reserve_datetime_diff','reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 
                        'reserve_datetime_diff':'rs1', 'reserve_visitors':'rv1'})
    tmp2 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[[
           'reserve_datetime_diff','reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 
                        'reserve_datetime_diff':'rs2', 'reserve_visitors':'rv2'})
    data[df] = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id', 'visit_date'])

In [84]:
data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date

data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

In [85]:
unique_stores = data['tes']['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)],
                   axis=0, ignore_index=True).reset_index(drop=True)

In [86]:
df_16 = data['tra'].loc[data['tra'].year==2016,:]
train_df = data['tra'].loc[data['tra'].year==2017,:] 

In [87]:
#sure it can be compressed...
store_dow_gpby = df_16.groupby(['air_store_id','dow'])['visitors']

tmp = store_dow_gpby.min().reset_index().rename(columns={'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 

tmp = store_dow_gpby.mean().reset_index().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])

tmp = store_dow_gpby.median().reset_index().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])

tmp = store_dow_gpby.max().reset_index().rename(columns={'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])

tmp = store_dow_gpby.count().reset_index().rename(columns={'visitors':'count_16_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 

tmp = store_dow_gpby.std().reset_index().rename(columns={'visitors':'std_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])

stores = pd.merge(stores, data['as'], how='left', on=['air_store_id'])
stores.head(2)

,air_store_id,dow,min_visitors,mean_visitors,median_visitors,max_visitors,count_16_visitors,std_visitors,air_genre_name,air_area_name,latitude,longitude
0,air_00a91d42b08b08d9,0,1.0,20.590909,19.0,47.0,22.0,8.743210,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595
1,air_0164b9927d20bcc3,0,2.0,6.888889,6.0,19.0,9.0,5.464532,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [88]:
stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))
stores['air_area_name'] = stores['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))
lbl = preprocessing.LabelEncoder()
for i in range(10):
    stores['air_genre_name'+str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
    stores['air_area_name'+str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])
stores.head(2)

,air_store_id,dow,min_visitors,mean_visitors,median_visitors,max_visitors,count_16_visitors,std_visitors,air_genre_name,air_area_name,...,air_genre_name5,air_area_name5,air_genre_name6,air_area_name6,air_genre_name7,air_area_name7,air_genre_name8,air_area_name8,air_genre_name9,air_area_name9
0,air_00a91d42b08b08d9,0,1.0,20.590909,19.0,47.0,22.0,8.743210,6,44,...,0,0,0,0,0,0,0,0,0,0
1,air_0164b9927d20bcc3,0,2.0,6.888889,6.0,19.0,9.0,5.464532,6,62,...,0,0,0,0,0,0,0,0,0,0


In [89]:
data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = data['hol'].visit_date.dt.dayofweek
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date
data['hol'].head(2)

,visit_date,day_of_week,holiday_flg
0,2016-01-01,4,1
1,2016-01-02,5,1


In [90]:
train = pd.merge(train_df, data['hol'], how='left', on=['visit_date']) 
test = pd.merge(data['tes'], data['hol'], how='left', on=['visit_date']) 

train = pd.merge(train, stores, how='left', on=['air_store_id','dow']) 
test = pd.merge(test, stores, how='left', on=['air_store_id','dow'])


In [91]:
for df in ['ar','hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])

print(train.shape, test.shape)

(77573, 46) (32019, 47)


In [114]:
train['total_reserv_sum'] = train['rv1_x'] + train['rv1_y']
train['total_reserv_mean'] = (train['rv2_x'] + train['rv2_y']) / 2
train['total_reserv_dt_diff_mean'] = (train['rs2_x'] + train['rs2_y']) / 2

test['total_reserv_sum'] = test['rv1_x'] + test['rv1_y']
test['total_reserv_mean'] = (test['rv2_x'] + test['rv2_y']) / 2
test['total_reserv_dt_diff_mean'] = (test['rs2_x'] + test['rs2_y']) / 2

In [115]:
train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']

# NEW FEATURES FROM Georgii Vyshnia
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

lbl = preprocessing.LabelEncoder()
train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
test['air_store_id2'] = lbl.transform(test['air_store_id'])


In [116]:
col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors']]
print(col)

['dow', 'year', 'month', 'day_of_week', 'holiday_flg', 'min_visitors', 'mean_visitors', 'median_visitors', 'max_visitors', 'count_16_visitors', 'std_visitors', 'air_genre_name', 'air_area_name', 'latitude', 'longitude', 'air_genre_name0', 'air_area_name0', 'air_genre_name1', 'air_area_name1', 'air_genre_name2', 'air_area_name2', 'air_genre_name3', 'air_area_name3', 'air_genre_name4', 'air_area_name4', 'air_genre_name5', 'air_area_name5', 'air_genre_name6', 'air_area_name6', 'air_genre_name7', 'air_area_name7', 'air_genre_name8', 'air_area_name8', 'air_genre_name9', 'air_area_name9', 'rs1_x', 'rv1_x', 'rs2_x', 'rv2_x', 'rs1_y', 'rv1_y', 'rs2_y', 'rv2_y', 'total_reserv_sum', 'total_reserv_mean', 'total_reserv_dt_diff_mean', 'date_int', 'var_max_lat', 'var_max_long', 'lon_plus_lat', 'air_store_id2']


In [117]:
train = train.fillna(-1)
test = test.fillna(-1)

In [119]:
def RMSE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

In [141]:
t0 = time.time()

model1 = LGBMRegressor(boosting_type='gbdt', num_leaves=60, max_depth=8, learning_rate=0.01, n_estimators=4500, max_bin=255, 
                       seed=42, nthread=-1)
model1.fit(train[col], np.log1p(train['visitors'].values))
preds1 = model1.predict(train[col])
print('Training of LGBMRegressor completed')
print('RMSE LGBMRegressor: ', RMSE(np.log1p(train['visitors'].values), preds1))
preds1 = model1.predict(test[col])

print(time_elapsed(t0))

Training of LGBMRegressor completed
RMSE LGBMRegressor:  0.41598208432838946
0.639 minutes elapsed!


In [142]:
# print("Features importance...")
# gain = lgb_model.feature_importance('gain')
# feat_imp = pd.DataFrame({'feature':lgb_model.feature_name(), 
#                          'split':lgb_model.feature_importance('split'), 
#                          'gain':100 * gain / gain.sum()}).sort_values('gain', ascending=False)
# print(feat_imp.head(50))

In [143]:
feat_imp = pd.DataFrame({'feature':col, 'imp':model1.feature_importances_}).sort_values('imp', ascending=False)
print(feat_imp)

                      feature    imp
46                   date_int  37560
50              air_store_id2  24227
10               std_visitors  19175
6               mean_visitors  17292
9           count_16_visitors  14753
8                max_visitors  14006
7             median_visitors  12536
24             air_area_name4  10161
5                min_visitors   9779
12              air_area_name   9317
11             air_genre_name   8111
0                         dow   8098
36                      rv1_x   7533
13                   latitude   7475
17            air_genre_name1   5549
14                  longitude   5442
20             air_area_name2   5438
35                      rs1_x   4477
38                      rv2_x   4033
37                      rs2_x   3831
49               lon_plus_lat   2883
2                       month   2496
42                      rv2_y   1726
40                      rv1_y   1555
4                 holiday_flg   1520
41                      rs2_y   1516
4

In [144]:
t0 = time.time()

model2 = neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)
model2.fit(train[col], np.log1p(train['visitors'].values))
preds2 = model2.predict(train[col])
print('Training of KNeighborsRegressor completed')
print('RMSE KNeighborsRegressor: ', RMSE(np.log1p(train['visitors'].values), preds2))
preds2 = model2.predict(test[col])

print(time_elapsed(t0))

Training of KNeighborsRegressor completed
RMSE KNeighborsRegressor:  0.41286349653957
0.269 minutes elapsed!


In [145]:
t0 = time.time()

model3 = ensemble.GradientBoostingRegressor(learning_rate=0.05, random_state=42, n_estimators=200, subsample=0.8, 
                                            max_depth=8, verbose=1)
model3.fit(train[col], np.log1p(train['visitors'].values))
preds3 = model3.predict(train[col])
print('Training of GradientBoostingRegressor completed')
print('RMSE GradientBoostingRegressor: ', RMSE(np.log1p(train['visitors'].values), preds3))
preds3 = model3.predict(test[col])

print(time_elapsed(t0))

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.6293           0.0382            4.71m
         2           0.5930           0.0350            4.74m
         3           0.5624           0.0309            4.68m
         4           0.5336           0.0283            4.66m
         5           0.5055           0.0255            4.65m
         6           0.4821           0.0238            4.64m
         7           0.4613           0.0213            4.63m
         8           0.4401           0.0193            4.62m
         9           0.4233           0.0176            4.61m
        10           0.4083           0.0160            4.60m
        20           0.3081           0.0059            4.43m
        30           0.2662           0.0024            4.17m
        40           0.2475           0.0010            3.93m
        50           0.2373           0.0005            3.64m
        60           0.2294           0.0004            3.35m
       

In [146]:
test['visitors'] = 0.3*preds1+0.3*preds2+0.4*preds3
test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)
test['visitors'].head()

0     5.155080
1    25.311924
2    28.272544
3    28.130924
4    32.037163
Name: visitors, dtype: float64

In [149]:
test[['id','visitors']].to_csv(os.path.join(BASE_DIR, 'submissions', 'bad_sub.csv'), index=False)

In [151]:
# Submit the submission on Kaggle
fname = 'bad_sub.csv'
config_cmd = 'kg submit '+os.path.join(BASE_DIR, 'submissions', fname)+' -c recruit-restaurant-visitor-forecasting'
os.system(config_cmd)

0